In [ ]:
!omz_downloader -h

In [ ]:
!omz_downloader --print_all  # 다운로드 가능한 모델들

In [ ]:
!omz_downloader --name face-detection-adas-0001 --precision FP16

## 기초 실습

In [1]:
import openvino as ov

In [2]:
core = ov.Core()

model = core.read_model(model="models/face-detection-adas-0001.xml")
face_model = core.compile_model(model=model, device_name="CPU")

face_input_layer = face_model.input(0)
face_output_layer = face_model.output(0)
print("Input layer shape:", face_input_layer.shape)
print("Output layer shape:", face_output_layer.shape)

Input layer shape: [1,3,384,672]
Output layer shape: [1,1,200,7]


In [3]:
import cv2
import numpy as np

image = cv2.imread("data/adults.jpg")
#print(image.shape)

resized_image = cv2.resize(src=image, dsize=(672,384))
transposed_image = resized_image.transpose(2,0,1)  # (384,672,3) => (3,384,672)
input_image = np.expand_dims(transposed_image, 0)  # 차원확장. (1,3,384,672)

In [4]:
face_output = face_model([input_image])[face_output_layer] # input_frame을 face_output_layer형태로 돌려주겠다.
print(face_output)  # 출력형태 : (image ID, label, conf, x_min, y_min, x_max, y_max)

[[[[0.         1.         1.         ... 0.1503618  0.6402889
    0.29027292]
   [0.         1.         1.         ... 0.24026465 0.3971492
    0.37072134]
   [0.         1.         1.         ... 0.25206834 0.7983894
    0.38078588]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]


In [ ]:
 # 넘파이 배열 자세히 보기
np.set_printoptions(threshold=np.inf, linewidth=np.inf)  # 넘파이 배열 자세히 예쁘게 보기
print(face_output)

In [ ]:
# 넘파이 배열 자세히 보기를 취소하고 원래 설정값으로 보기
np.set_printoptions(edgeitems=3, infstr='inf', linewidth=75, nanstr='nan', precision=8, suppress=False, threshold=1000, formatter=None)

In [5]:
def DrawBoundingBoxes(output, image, conf):
    canvas = image.copy()                             # 원본 이미지를 수정하는 대신 복사합니다
    h,w,_ = canvas.shape                      # 이미지의 형태 : (높이, 폭, 채널)
    
    predictions = output[0][0]                         # 하위 집합 데이터 프레임
    confidence = predictions[:,2]                       # conf 값 가져오기 [image_id, label, conf, x_min, y_min, x_max, y_max]
  
    top_predictions = predictions[(confidence>conf)]         # 임계값보다 큰 conf 값을 가진 예측만 선택
 
    for detection in top_predictions:
        box = detection[3:7] * np.array([w, h, w, h]) # 상자 위치 결정 
        (xmin, ymin, xmax, ymax) = box.astype("int")   # xmin, ymin, xmax, ymax에 상자 위치 값 지정(자료형을 int로 변경)
   
        cv2.rectangle(canvas, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)       # 사각형 만들기
    
    return canvas

In [6]:
canvas = DrawBoundingBoxes(face_output, image, conf=0.5)
cv2.imshow("Face", canvas) # 이미지 보기

cv2.waitKey(0)           # 키입력 대기(ms 단위). 0이면 무한대기. waitKey()함수가 없다면 imshow()함수는 순식간에 지나가서 볼 수가 없음.
cv2.destroyAllWindows()  # 열린 모든 창 닫기

### 배경화면에 이미지 넣기

In [19]:
def AddBackground(frame, bg):

    frame_h, frame_w = frame.shape[0], frame.shape[1]
    new_h = 500
    new_w = int((new_h/frame_h)*frame_w)
    frame_resize = cv2.resize(frame, (new_w, new_h))

    xmax = bg.shape[1] - 270
    ymax = bg.shape[0] - 175
    xmin = xmax - new_w
    ymin = ymax - new_h

    bg[ymin:ymax, xmin:xmax] = frame_resize

    return bg

In [20]:
background = "data/background.jpg"  #사용할 배경화면 경로
canvas = DrawBoundingBoxes(face_output, image, conf=0.5)  
bg = cv2.imread(background)

deployment = AddBackground(canvas, bg)
cv2.imshow("Deployment", deployment)

cv2.waitKey(0)
cv2.destroyAllWindows()

## 배포(Gradio)

In [22]:
import openvino as ov
import cv2
import numpy as np
import gradio as gr

In [23]:
core = ov.Core()
options = core.available_devices
options

['CPU', 'GPU']

In [24]:
model = core.read_model(model="models/face-detection-adas-0001.xml")
face_model = core.compile_model(model=model, device_name="CPU")

face_input_layer = face_model.input(0)
face_output_layer = face_model.output(0)

In [25]:
# 새로 입력된 이미지 데이터 전처리
def preprocess(new_image):
    resized_image = cv2.resize(src=new_image, dsize=(672, 384)) 
    transposed_image = resized_image.transpose(2, 0, 1)
    input_image = np.expand_dims(transposed_image, 0)
    
    return input_image

In [26]:
# AI 추론 결과 후처리: 시각화(인식된 얼굴 주변에 박스 그리기)
def DrawBoundingBoxes(new_image, face_output, conf):
    canvas = new_image.copy()
    h,w,_ = canvas.shape 
    predictions = face_output[0][0]            # 하위 집합 데이터 프레임
    confidence = predictions[:,2]              # conf 값 가져오기 [image_id, label, conf, x_min, y_min, x_max, y_max]
    top_predictions = predictions[(confidence>conf)]         # 임계값보다 큰 conf 값을 가진 예측만 선택
    for detection in top_predictions:
        box = detection[3:7] * np.array([w, h, w, h]) # 상자 위치 결정
        (xmin, ymin, xmax, ymax) = box.astype("int")  # xmin, ymin, xmax, ymax에 상자 위치 값 지정
        cv2.rectangle(canvas, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)       # 사각형 만들기
        
    return canvas

In [27]:
# AI 추론
def predict_image(new_image):
    input_image = preprocess(new_image)  # Preprocess the image
    face_output = face_model([input_image])[face_output_layer]  # Perform inference  
    canvas = DrawBoundingBoxes(new_image, face_output, conf=0.5)
    
    return canvas

In [28]:
gr.Interface(fn=predict_image,
             inputs=gr.Image(type="numpy"),  # 넘파이 배열 사용
             outputs=gr.Image(type="numpy"), # 넘파이 배열 처럼 출력
             examples=["data/test.jpg"]).launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## 영상 추론

In [30]:
camera = cv2.VideoCapture(0)   # 웹캠의 경우는 0, 동영상의 경우는 경로 적기

while(True):
    ret, frame = camera.read()
    if ret == False:
        break

    face_detection = predict_image(frame)
    cv2.imshow('Press Spacebar to Exit', face_detection)

    if cv2.waitKey(1) & 0xFF == ord(' '):
        break

camera.release()
cv2.destroyAllWindows()

In [29]:
# 배경 화면에 영상 넣기
camera = cv2.VideoCapture(0)   # 웹캠의 경우는 0, 동영상의 경우는 경로 적기
background = "data/background.jpg"  #사용할 배경화면 경로

while(True):
    ret, frame = camera.read()
    if ret == False:
        break

    face_detection = predict_image(frame)
    bg = cv2.imread(background)

    deployment = AddBackground(face_detection, bg)
    cv2.imshow('Press Spacebar to Exit', deployment)

    if cv2.waitKey(1) & 0xFF == ord(' '):
        break

camera.release()
cv2.destroyAllWindows()